In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from lmfit.models import GaussianModel, LinearModel, ExponentialModel, Model

/home/arrow13/anaconda3/lib/python3.4/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


# Kalibrierung

In [2]:
untergrund = pd.read_csv("Messdaten/kalibrierung/kallibrierung_untergrund.csv", header=14, delim_whitespace=True, names=['rate','error'], index_col=0)
am = pd.read_csv("Messdaten/kalibrierung/kallibrierung_Am.csv", header=14, delim_whitespace=True, names=['rate','error'], index_col=0)
ba = pd.read_csv("Messdaten/kalibrierung/kallibrierung_Ba.csv", header=14, delim_whitespace=True, names=['rate','error'], index_col=0)
cs = pd.read_csv("Messdaten/kalibrierung/kallibrierung_Cs.csv", header=14, delim_whitespace=True, names=['rate','error'], index_col=0)

In [3]:
x = np.arange(0,2048)

In [4]:
plt.errorbar(x,untergrund.rate,yerr=untergrund.error, fmt=' .g')
plt.xlim([0,2050])
plt.xlabel("Kanal",fontsize=24)
plt.ylabel("Counts",fontsize=24)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.legend(loc='best',fontsize=24)
plt.show()

/home/arrow13/anaconda3/lib/python3.4/site-packages/matplotlib/axes/_axes.py:475: UserWarning: No labelled objects found. Use label='...' kwarg on individual plots.
  warnings.warn("No labelled objects found. "


In [5]:
plt.plot(am.rate-untergrund.rate, ' .', label="Messdaten")
plt.legend(loc="best")
plt.show()

In [6]:
plt.plot(ba.rate-untergrund.rate, ' .')
plt.show()

In [7]:
plt.plot(cs.rate-untergrund.rate, ' .')
plt.show()

## Fit der Am Daten

In [8]:
x = np.arange(0,2048)

In [9]:
gauss_0 = GaussianModel(prefix="gauss_0_")
gauss_1 = GaussianModel(prefix="gauss_1_")
gauss_2 = GaussianModel(prefix="gauss_2_")
gauss_3 = GaussianModel(prefix="gauss_3_")
gauss_4 = GaussianModel(prefix="gauss_4_")
gauss_5 = GaussianModel(prefix="gauss_5_")
gauss_6 = GaussianModel(prefix="gauss_6_")

In [10]:
pars_am_1 = gauss_1.guess(am.rate,x=x)
pars_am_1.update(gauss_2.make_params())
pars_am_1.update(gauss_3.make_params())
pars_am_1.update(gauss_4.make_params())
pars_am_1.update(gauss_0.make_params())
pars_am_2 = gauss_5.guess(am.rate,x=x)
pars_am_2.update(gauss_6.make_params())

In [11]:
# controll the center
pars_am_1['gauss_0_center'].set(50, min=48, max=52)
pars_am_1['gauss_1_center'].set(65, min=63, max=68)
pars_am_1['gauss_2_center'].set(77, min=74, max=80)
pars_am_1['gauss_3_center'].set(98, min=95, max=101)
pars_am_1['gauss_4_center'].set(111, min=108, max=114)
pars_am_2['gauss_5_center'].set(181, min=178, max=184)
pars_am_2['gauss_6_center'].set(213, min=207, max=219)
# controll the amplitude
pars_am_1['gauss_0_amplitude'].set(130*13)#, min=48, max=52)
pars_am_1['gauss_1_amplitude'].set(440*13)#, min=430*2.5, max=440*2.5)
pars_am_1['gauss_2_amplitude'].set(300*13)#, min=290*2.5, max=310*2.5)
pars_am_1['gauss_3_amplitude'].set(310*13)#, min=300*2.5, max=320*2.5)
pars_am_1['gauss_4_amplitude'].set(380*13)#, min=370*2.5, max=390*2.5)
pars_am_2['gauss_5_amplitude'].set(500*13)#, min=490*2.5, max=510*2.5)
pars_am_2['gauss_6_amplitude'].set(2500*13)#, min=2400*2.5, max=2600*2.5)
# controll sigma
pars_am_1['gauss_0_sigma'].set(1, min=0, max=5)
pars_am_1['gauss_1_sigma'].set(5, min=1, max=68)
pars_am_1['gauss_2_sigma'].set(5, min=1, max=80)
pars_am_1['gauss_3_sigma'].set(5, min=1, max=101)
pars_am_1['gauss_4_sigma'].set(5, min=1, max=114)
pars_am_2['gauss_5_sigma'].set(5, min=1, max=184)
pars_am_2['gauss_6_sigma'].set(5, min=1, max=219)

In [12]:
mod_gauss_am_1 = gauss_1 + gauss_2 + gauss_3 + gauss_4 + gauss_0

In [13]:
out_gauss_am_1 = mod_gauss_am_1.fit(am.rate[40:130]-untergrund.rate[40:130],x=x[40:130],params=pars_am_1,weights=1/(am.error[40:130]))

In [14]:
plt.plot(x,out_gauss_am_1.eval(x=np.arange(0,2048)), '-g')
plt.errorbar(x,am.rate-untergrund.rate, yerr=am.error, fmt=' .r')
plt.show()

In [15]:
print(out_gauss_am_1.fit_report())

[[Model]]
    ((((Model(gaussian, prefix='gauss_1_') + Model(gaussian, prefix='gauss_2_')) + Model(gaussian, prefix='gauss_3_')) + Model(gaussian, prefix='gauss_4_')) + Model(gaussian, prefix='gauss_0_'))
[[Fit Statistics]]
    # function evals   = 215
    # data points      = 90
    # variables        = 15
    chi-square         = 86.581
    reduced chi-square = 1.154
[[Variables]]
    gauss_1_center:      64.5491639 +/- 0.452557 (0.70%) (init= 65)
    gauss_1_amplitude:   4871.27167 +/- 649.3600 (13.33%) (init= 5720)
    gauss_1_sigma:       4.84841725 +/- 0.444423 (9.17%) (init= 5)
    gauss_1_fwhm:        11.4171499 +/- 1.046538 (9.17%)  == '2.3548200*gauss_1_sigma'
    gauss_2_center:      77.2652555 +/- 0.717170 (0.93%) (init= 77)
    gauss_2_amplitude:   3939.85307 +/- 652.0800 (16.55%) (init= 3900)
    gauss_2_sigma:       5.50873442 +/- 0.729373 (13.24%) (init= 5)
    gauss_2_fwhm:        12.9720780 +/- 1.717542 (13.24%)  == '2.3548200*gauss_2_sigma'
    gauss_3_center:      9

In [16]:
mod_gauss_am_2 = gauss_5 + gauss_6

In [17]:
out_gauss_am_2 = mod_gauss_am_2.fit(am.rate[165:240]-untergrund.rate[165:240],x=x[165:240],params=pars_am_2,weights=1/am.error[165:240])

In [18]:
plt.plot(x,out_gauss_am_2.eval(x=x), '-g')
plt.errorbar(x,am.rate-untergrund.rate, yerr=am.error, fmt=' .r')
plt.show()

In [19]:
print(out_gauss_am_2.fit_report())

[[Model]]
    (Model(gaussian, prefix='gauss_5_') + Model(gaussian, prefix='gauss_6_'))
[[Fit Statistics]]
    # function evals   = 99
    # data points      = 75
    # variables        = 6
    chi-square         = 115.754
    reduced chi-square = 1.678
[[Variables]]
    gauss_5_sigma:       13.6749514 +/- 0        (0.00%) (init= 5)
    gauss_5_center:      184        +/- 0        (0.00%) (init= 181)
    gauss_5_amplitude:   16761.1470 +/- 0        (0.00%) (init= 6500)
    gauss_5_fwhm:        32.2020491 +/- 0        (0.00%)  == '2.3548200*gauss_5_sigma'
    gauss_6_sigma:       8.56075034 +/- 0        (0.00%) (init= 5)
    gauss_6_center:      213.627553 +/- 0        (0.00%) (init= 213)
    gauss_6_amplitude:   52316.5173 +/- 0        (0.00%) (init= 32500)
    gauss_6_fwhm:        20.1590261 +/- 0        (0.00%)  == '2.3548200*gauss_6_sigma'
[[Correlations]] (unreported correlations are <  0.100)


## Fit der Ba Daten

In [20]:
gauss_0 = GaussianModel(prefix="gauss_0_")
gauss_1 = GaussianModel(prefix="gauss_1_")
gauss_2 = GaussianModel(prefix="gauss_2_")
gauss_3 = GaussianModel(prefix="gauss_3_")
gauss_4 = GaussianModel(prefix="gauss_4_")
gauss_5 = GaussianModel(prefix="gauss_5_")
gauss_6 = GaussianModel(prefix="gauss_6_")

In [21]:
pars_ba_1 = gauss_1.guess(ba.rate,x=x)
pars_ba_2 = gauss_2.guess(ba.rate,x=x)
pars_ba_2.update(gauss_0.make_params())
pars_ba_3 = gauss_3.guess(ba.rate,x=x)
pars_ba_3.update(gauss_4.make_params())
pars_ba_4 = gauss_5.guess(ba.rate,x=x)

In [22]:
# controll the center
pars_ba_1['gauss_1_center'].set(20, min=10, max=25)
pars_ba_2['gauss_2_center'].set(112, min=100, max=120)
pars_ba_2['gauss_0_center'].set(125, min=120, max=130)
pars_ba_3['gauss_3_center'].set(228, min=210, max=235)
pars_ba_3['gauss_4_center'].set(286, min=270, max=300)
pars_ba_4['gauss_5_center'].set(1182, min=1150, max=1200)
# controll the amplitude
pars_ba_1['gauss_1_amplitude'].set(440*13, min=0)#, max=440*2.5)
pars_ba_2['gauss_2_amplitude'].set(300, min=0)#, max=310*2.5)
pars_ba_2['gauss_0_amplitude'].set(125, min=0)#, max=130)
pars_ba_3['gauss_3_amplitude'].set(310*13, min=0)#, max=320*2.5)
pars_ba_3['gauss_4_amplitude'].set(380*13, min=0)#, max=390*2.5)
pars_ba_4['gauss_5_amplitude'].set(500*13, min=0)#, max=510*2.5)
# controll sigma
pars_ba_1['gauss_1_sigma'].set(5, min=1, max=68)
pars_ba_2['gauss_2_sigma'].set(5, min=1, max=80)
pars_ba_3['gauss_3_sigma'].set(5, min=1, max=101)
pars_ba_3['gauss_4_sigma'].set(5, min=1, max=114)
pars_ba_4['gauss_5_sigma'].set(5, min=1, max=184)

In [23]:
mod_ba_1 = gauss_1
out_ba_1 = mod_ba_1.fit(ba.rate[15:25]-untergrund.rate[15:25],params=pars_ba_1,x=x[15:25],weights=1/ba.error[15:25])

In [24]:
plt.plot(x,out_ba_1.eval(x=x), '-g')
plt.errorbar(x,ba.rate-untergrund.rate, yerr=ba.error, fmt=' .r')
plt.show()

In [25]:
print(out_ba_1.fit_report())

[[Model]]
    Model(gaussian, prefix='gauss_1_')
[[Fit Statistics]]
    # function evals   = 25
    # data points      = 10
    # variables        = 3
    chi-square         = 7.968
    reduced chi-square = 1.138
[[Variables]]
    gauss_1_center:      19.1213071 +/- 0.201517 (1.05%) (init= 20)
    gauss_1_amplitude:   1117.53081 +/- 61.21242 (5.48%) (init= 5720)
    gauss_1_sigma:       3.71202384 +/- 0.281089 (7.57%) (init= 5)
    gauss_1_fwhm:        8.74114799 +/- 0.661916 (7.57%)  == '2.3548200*gauss_1_sigma'
[[Correlations]] (unreported correlations are <  0.100)
    C(gauss_1_amplitude, gauss_1_sigma)  =  0.751 
    C(gauss_1_center, gauss_1_sigma)  = -0.209 
    C(gauss_1_center, gauss_1_amplitude)  = -0.204 


In [26]:
mod_ba_2 = gauss_2 + gauss_0
out_ba_2 = mod_ba_2.fit(ba.rate[95:135]-untergrund.rate[95:135],params=pars_ba_2,x=x[95:135],weights=1/ba.error[95:135])

In [27]:
plt.plot(x,out_ba_2.eval(x=x), '-g')
plt.errorbar(x,ba.rate-untergrund.rate, yerr=ba.error, fmt=' .r')
plt.show()

In [28]:
print(out_ba_2.fit_report())

[[Model]]
    (Model(gaussian, prefix='gauss_2_') + Model(gaussian, prefix='gauss_0_'))
[[Fit Statistics]]
    # function evals   = 132
    # data points      = 40
    # variables        = 6
    chi-square         = 80.259
    reduced chi-square = 2.361
[[Variables]]
    gauss_2_center:      111.053939 +/- 0.126139 (0.11%) (init= 112)
    gauss_2_amplitude:   26863.5421 +/- 382.4151 (1.42%) (init= 300)
    gauss_2_sigma:       6.78656619 +/- 0.105521 (1.55%) (init= 5)
    gauss_2_fwhm:        15.9811418 +/- 0.248484 (1.55%)  == '2.3548200*gauss_2_sigma'
    gauss_0_center:      128.025141 +/- 0.499980 (0.39%) (init= 125)
    gauss_0_amplitude:   2615.75297 +/- 323.1183 (12.35%) (init= 125)
    gauss_0_sigma:       4.59849939 +/- 0.499771 (10.87%) (init= 1)
    gauss_0_fwhm:        10.8286383 +/- 1.176872 (10.87%)  == '2.3548200*gauss_0_sigma'
[[Correlations]] (unreported correlations are <  0.100)
    C(gauss_0_amplitude, gauss_0_sigma)  =  0.864 
    C(gauss_0_center, gauss_0_amplitud

In [29]:
mod_ba_3 = gauss_3 + gauss_4
out_ba_3 = mod_ba_3.fit(ba.rate[204:315]-untergrund.rate[204:315],params=pars_ba_3,x=x[204:315],weights=1/ba.error[204:315])

In [30]:
plt.plot(x,out_ba_3.eval(x=x), '-g')
plt.errorbar(x,ba.rate-untergrund.rate, yerr=ba.error, fmt=' .r')
plt.show()

In [31]:
print(out_ba_3.fit_report())

[[Model]]
    (Model(gaussian, prefix='gauss_3_') + Model(gaussian, prefix='gauss_4_'))
[[Fit Statistics]]
    # function evals   = 77
    # data points      = 111
    # variables        = 6
    chi-square         = 186.847
    reduced chi-square = 1.779
[[Variables]]
    gauss_3_center:      234.999999 +/- 5.343194 (2.27%) (init= 228)
    gauss_3_sigma:       29.2420710 +/- 3.553439 (12.15%) (init= 5)
    gauss_3_amplitude:   7263.68068 +/- 781.6791 (10.76%) (init= 4030)
    gauss_3_fwhm:        68.8598136 +/- 8.367709 (12.15%)  == '2.3548200*gauss_3_sigma'
    gauss_4_sigma:       12.5318055 +/- 0.343816 (2.74%) (init= 5)
    gauss_4_center:      284.429428 +/- 0.300667 (0.11%) (init= 286)
    gauss_4_amplitude:   11357.4608 +/- 560.8607 (4.94%) (init= 4940)
    gauss_4_fwhm:        29.5101462 +/- 0.809626 (2.74%)  == '2.3548200*gauss_4_sigma'
[[Correlations]] (unreported correlations are <  0.100)
    C(gauss_3_sigma, gauss_3_amplitude)  =  0.971 
    C(gauss_3_amplitude, gauss_4_am

In [32]:
mod_ba_4 = gauss_5
out_ba_4 = mod_ba_4.fit(ba.rate[1100:1250],params=pars_ba_4,x=x[1100:1250],weights=1/ba.error[1100:1250])

In [33]:
plt.plot(x,out_ba_4.eval(x=x), '-g')
plt.errorbar(x,ba.rate-untergrund.rate, yerr=ba.error, fmt=' .r')
plt.show()

In [34]:
print(out_ba_4.fit_report())

[[Model]]
    Model(gaussian, prefix='gauss_5_')
[[Fit Statistics]]
    # function evals   = 33
    # data points      = 150
    # variables        = 3
    chi-square         = 117.888
    reduced chi-square = 0.802
[[Variables]]
    gauss_5_sigma:       43.3949452 +/- 0.494619 (1.14%) (init= 5)
    gauss_5_center:      1180.46864 +/- 0.450395 (0.04%) (init= 1182)
    gauss_5_amplitude:   12789.9828 +/- 117.5778 (0.92%) (init= 6500)
    gauss_5_fwhm:        102.187285 +/- 1.164740 (1.14%)  == '2.3548200*gauss_5_sigma'
[[Correlations]] (unreported correlations are <  0.100)
    C(gauss_5_sigma, gauss_5_amplitude)  =  0.430 
    C(gauss_5_sigma, gauss_5_center)  =  0.180 
    C(gauss_5_center, gauss_5_amplitude)  =  0.128 


## Kilibrierung der Cs Daten

In [35]:
gauss_0 = GaussianModel(prefix="gauss_0_")
gauss_1 = GaussianModel(prefix="gauss_1_")
gauss_2 = GaussianModel(prefix="gauss_2_")

In [36]:
pars_cs_1 = gauss_0.guess(cs.rate,x=x)
pars_cs_2 = gauss_1.guess(cs.rate,x=x)
pars_cs_3 = gauss_2.guess(cs.rate,x=x)

In [37]:
# controll the center
pars_cs_1['gauss_0_center'].set(115, min=110, max=120)
pars_cs_2['gauss_1_center'].set(267, min=263, max=270)
pars_cs_3['gauss_2_center'].set(1920, min=1900, max=1940)
# controll the amplitude
pars_cs_1['gauss_0_amplitude'].set(125, min=0)#, max=130)
pars_cs_2['gauss_1_amplitude'].set(440*13, min=0)#, max=440*2.5)
pars_cs_3['gauss_2_amplitude'].set(300, min=0)#, max=310*2.5)
# controll sigma
pars_cs_1['gauss_0_sigma'].set(5, min=1, max=68)
pars_cs_2['gauss_1_sigma'].set(5, min=1, max=68)
pars_cs_3['gauss_2_sigma'].set(5, min=1, max=80)

In [38]:
mod_cs_1 = gauss_0
out_cs_1 = mod_cs_1.fit(cs.rate[100:130],params=pars_cs_1,x=x[100:130],weights=1/cs.error[100:130])

In [39]:
plt.plot(x,out_cs_1.eval(x=x), '-g')
plt.errorbar(x,cs.rate-untergrund.rate, yerr=cs.error, fmt=' .r')
plt.show()

In [40]:
print(out_cs_1.fit_report())

[[Model]]
    Model(gaussian, prefix='gauss_0_')
[[Fit Statistics]]
    # function evals   = 35
    # data points      = 30
    # variables        = 3
    chi-square         = 36.299
    reduced chi-square = 1.344
[[Variables]]
    gauss_0_center:      115.366805 +/- 0.153160 (0.13%) (init= 115)
    gauss_0_amplitude:   6336.70331 +/- 103.7750 (1.64%) (init= 125)
    gauss_0_sigma:       8.33786025 +/- 0.166957 (2.00%) (init= 5)
    gauss_0_fwhm:        19.6341600 +/- 0.393154 (2.00%)  == '2.3548200*gauss_0_sigma'
[[Correlations]] (unreported correlations are <  0.100)
    C(gauss_0_amplitude, gauss_0_sigma)  =  0.381 
    C(gauss_0_center, gauss_0_sigma)  =  0.137 


In [41]:
mod_cs_3 = gauss_2
out_cs_3 = mod_cs_3.fit(cs.rate[1860:1980],params=pars_cs_3,x=x[1860:1980],weights=1/cs.error[1860:1980])

In [42]:
plt.errorbar(x,cs.rate-untergrund.rate, yerr=cs.error, fmt=' .g', label="Messdaten")
plt.plot(x,out_cs_3.eval(x=x), '-r', label="Fit")
plt.xlabel("Kanal",fontsize=24)
plt.ylabel("Counts",fontsize=24)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.legend(loc='best',fontsize=24)
plt.show()

In [43]:
print(out_cs_3.fit_report())

[[Model]]
    Model(gaussian, prefix='gauss_2_')
[[Fit Statistics]]
    # function evals   = 29
    # data points      = 120
    # variables        = 3
    chi-square         = 170.233
    reduced chi-square = 1.455
[[Variables]]
    gauss_2_center:      1920.26351 +/- 0.187739 (0.01%) (init= 1920)
    gauss_2_amplitude:   23839.7666 +/- 187.4390 (0.79%) (init= 300)
    gauss_2_sigma:       23.0352370 +/- 0.152370 (0.66%) (init= 5)
    gauss_2_fwhm:        54.2438368 +/- 0.358805 (0.66%)  == '2.3548200*gauss_2_sigma'
[[Correlations]] (unreported correlations are <  0.100)


In [44]:
energien = np.array([14,33,60,31,81,383,32,662])
kanal = np.array([64.5491642,112.029958,213.627553,111.053940,284.659378,1180.46864,115.366805,1920.26351])
kanal_err = np.array([4.84841741,6.34840458,8.56075037,6.78656645,12.5318055,43.3949452,8.33786026,23.0352370])

In [45]:
plt.errorbar(kanal,energien,yerr=kanal_err, fmt=' .g')
plt.show()

In [46]:
def lin_fit(x,m,b):
    return (x-b)/m

In [47]:
mod_lin = Model(lin_fit)
pars_lin = mod_lin.make_params(m=1,b=1)
out_lin = mod_lin.fit(kanal,params=pars_lin,x=energien,weights=1/(kanal_err))

In [48]:
out_lin.plot()
plt.legend(loc="best")
plt.show()

In [49]:
print(out_lin.fit_report())

[[Model]]
    Model(lin_fit)
[[Fit Statistics]]
    # function evals   = 20
    # data points      = 8
    # variables        = 2
    chi-square         = 11.277
    reduced chi-square = 1.880
[[Variables]]
    b:  -8.63436257 +/- 1.575031 (18.24%) (init= 1)
    m:   0.34572897 +/- 0.005655 (1.64%) (init= 1)
[[Correlations]] (unreported correlations are <  0.100)
    C(b, m)                      = -0.530 


In [50]:
plt.errorbar(energien,kanal, yerr=kanal_err, fmt=' .g', label="Bestimmte Kanalpositionen")
plt.plot(np.arange(0,700),out_lin.eval(x=np.arange(0,700)), '-r', label="Fit")
plt.xlabel("Counts",fontsize=24)
plt.ylabel("Kanal",fontsize=24)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.legend(loc='best',fontsize=24)
plt.show()

# Absorbtion

In [51]:
def conv_to_counts(amplitude):
    return amplitude/(300*0.5)

In [52]:
def exp_fit(x,a,b):
    return a*np.exp(-b*x)

## Blei

In [53]:
blei_7cm = pd.read_csv("Messdaten/absorbtion/absorbtion_7cm.csv", header=14, delim_whitespace=True, names=['rate','error'], index_col=0)
blei_6cm = pd.read_csv("Messdaten/absorbtion/absorbtion_6cm.csv", header=14, delim_whitespace=True, names=['rate','error'], index_col=0)
blei_5cm= pd.read_csv("Messdaten/absorbtion/absorbtion_5cm.csv", header=14, delim_whitespace=True, names=['rate','error'], index_col=0)
blei_4_5cm= pd.read_csv("Messdaten/absorbtion/absorbtion_4,5cm_LLDhoch.csv", header=14, delim_whitespace=True, names=['rate','error'], index_col=0)
blei_3_5cm = pd.read_csv("Messdaten/absorbtion/absorbtion_3,5cm_LLDhoch.csv", header=14, delim_whitespace=True, names=['rate','error'], index_col=0)
blei_2_5cm= pd.read_csv("Messdaten/absorbtion/absorbtion_2,5cm_LLDhoch.csv", header=14, delim_whitespace=True, names=['rate','error'], index_col=0)

### 7cm

In [54]:
pars_absorb_7cm = gauss_1.guess(blei_7cm.rate-untergrund.rate,x=x)
# controll the center
pars_absorb_7cm['gauss_1_center'].set(1900, min=1800, max=2000)
# controll the amplitude
pars_absorb_7cm['gauss_1_amplitude'].set(440*13, min=0)#, max=440*2.5)
# controll sigma
pars_absorb_7cm['gauss_1_sigma'].set(5, min=1, max=68)

In [55]:
out_absorb_7cm = gauss_1.fit(blei_7cm.rate[1850:2000]-untergrund.rate[1850:2000],params=pars_absorb_7cm,x=x[1850:2000],weights=1/blei_7cm.error[1850:2000])

In [56]:
plt.errorbar(x,blei_7cm.rate-untergrund.rate, yerr=blei_7cm.error, fmt=' .g')
plt.plot(x,out_absorb_7cm.eval(x=x), '-r')
plt.show()

In [57]:
print(out_absorb_7cm.fit_report())

[[Model]]
    Model(gaussian, prefix='gauss_1_')
[[Fit Statistics]]
    # function evals   = 45
    # data points      = 150
    # variables        = 3
    chi-square         = 299.499
    reduced chi-square = 2.037
[[Variables]]
    gauss_1_center:      1922.65512 +/- 0.768987 (0.04%) (init= 1900)
    gauss_1_amplitude:   2284.71175 +/- 77.00543 (3.37%) (init= 5720)
    gauss_1_fwhm:        48.2671964 +/- 1.460439 (3.03%)  == '2.3548200*gauss_1_sigma'
    gauss_1_sigma:       20.4971914 +/- 0.620191 (3.03%) (init= 5)
[[Correlations]] (unreported correlations are <  0.100)
    C(gauss_1_amplitude, gauss_1_sigma)  =  0.199 


### 6cm

In [58]:
pars_absorb_6cm = gauss_1.guess(blei_6cm.rate-untergrund.rate,x=x)
# controll the center
pars_absorb_6cm['gauss_1_center'].set(1925, min=1900, max=2000)
# controll the amplitude
pars_absorb_6cm['gauss_1_amplitude'].set(440*18, min=0)#, max=440*2.5)
# controll sigma
pars_absorb_6cm['gauss_1_sigma'].set(20, min=1, max=68)

In [59]:
out_absorb_6cm = gauss_1.fit(blei_6cm.rate[1850:1993]-untergrund.rate[1850:1993],params=pars_absorb_6cm,x=x[1850:1993],weights=1/blei_6cm.error[1850:1993])

In [60]:
plt.errorbar(x,blei_6cm.rate-untergrund.rate, yerr=blei_6cm.error, fmt=' .g')
plt.plot(x,out_absorb_6cm.eval(x=x), '-r')
plt.show()

In [61]:
print(out_absorb_6cm.fit_report())

[[Model]]
    Model(gaussian, prefix='gauss_1_')
[[Fit Statistics]]
    # function evals   = 21
    # data points      = 143
    # variables        = 3
    chi-square         = 161.311
    reduced chi-square = 1.152
[[Variables]]
    gauss_1_center:      1922.16220 +/- 0.283142 (0.01%) (init= 1925)
    gauss_1_amplitude:   8299.78978 +/- 101.5909 (1.22%) (init= 7920)
    gauss_1_fwhm:        51.6917021 +/- 0.517111 (1.00%)  == '2.3548200*gauss_1_sigma'
    gauss_1_sigma:       21.9514451 +/- 0.219597 (1.00%) (init= 20)
[[Correlations]] (unreported correlations are <  0.100)


### 5cm

In [62]:
pars_absorb_5cm = gauss_1.guess(blei_5cm.rate-untergrund.rate,x=x)
# controll the center
pars_absorb_5cm['gauss_1_center'].set(1925, min=1900, max=2000)
# controll the amplitude
pars_absorb_5cm['gauss_1_amplitude'].set(440*18, min=0)#, max=440*2.5)
# controll sigma
pars_absorb_5cm['gauss_1_sigma'].set(20, min=1, max=68)

In [63]:
out_absorb_5cm = gauss_1.fit(blei_5cm.rate[1850:1993]-untergrund.rate[1850:1993],params=pars_absorb_5cm,x=x[1850:1993],weights=1/blei_5cm.error[1850:1993])

In [64]:
plt.errorbar(x,blei_5cm.rate-untergrund.rate, yerr=blei_5cm.error, fmt=' .g')
plt.plot(x,out_absorb_5cm.eval(x=x), '-r')
plt.show()

In [65]:
print(out_absorb_5cm.fit_report())

[[Model]]
    Model(gaussian, prefix='gauss_1_')
[[Fit Statistics]]
    # function evals   = 29
    # data points      = 143
    # variables        = 3
    chi-square         = 243.761
    reduced chi-square = 1.741
[[Variables]]
    gauss_1_center:      1921.75060 +/- 0.186295 (0.01%) (init= 1925)
    gauss_1_amplitude:   27339.4031 +/- 220.9497 (0.81%) (init= 7920)
    gauss_1_fwhm:        52.8151919 +/- 0.334281 (0.63%)  == '2.3548200*gauss_1_sigma'
    gauss_1_sigma:       22.4285473 +/- 0.141956 (0.63%) (init= 20)
[[Correlations]] (unreported correlations are <  0.100)


### 4,5cm

In [66]:
pars_absorb_4_5cm = gauss_1.guess(blei_4_5cm.rate-untergrund.rate,x=x)
# controll the center
pars_absorb_4_5cm['gauss_1_center'].set(1925, min=1900, max=2000)
# controll the amplitude
pars_absorb_4_5cm['gauss_1_amplitude'].set(440*18, min=0)#, max=440*2.5)
# controll sigma
pars_absorb_4_5cm['gauss_1_sigma'].set(20, min=1, max=68)

In [67]:
out_absorb_4_5cm = gauss_1.fit(blei_4_5cm.rate[1850:1993]-untergrund.rate[1850:1993],params=pars_absorb_4_5cm,x=x[1850:1993],weights=1/blei_4_5cm.error[1850:1993])

In [68]:
plt.errorbar(x,blei_4_5cm.rate-untergrund.rate, yerr=blei_4_5cm.error, fmt=' .g')
plt.plot(x,out_absorb_4_5cm.eval(x=x), '-r')
plt.show()

In [69]:
print(out_absorb_4_5cm.fit_report())

[[Model]]
    Model(gaussian, prefix='gauss_1_')
[[Fit Statistics]]
    # function evals   = 37
    # data points      = 143
    # variables        = 3
    chi-square         = 352.808
    reduced chi-square = 2.520
[[Variables]]
    gauss_1_center:      1918.16323 +/- 0.167978 (0.01%) (init= 1925)
    gauss_1_amplitude:   48414.7069 +/- 352.0339 (0.73%) (init= 7920)
    gauss_1_fwhm:        53.1014941 +/- 0.297242 (0.56%)  == '2.3548200*gauss_1_sigma'
    gauss_1_sigma:       22.5501287 +/- 0.126227 (0.56%) (init= 20)
[[Correlations]] (unreported correlations are <  0.100)
    C(gauss_1_center, gauss_1_sigma)  = -0.103 


### 3,5cm

In [70]:
pars_absorb_3_5cm = gauss_1.guess(blei_3_5cm.rate-untergrund.rate,x=x)
# controll the center
pars_absorb_3_5cm['gauss_1_center'].set(1925, min=1900, max=2000)
# controll the amplitude
pars_absorb_3_5cm['gauss_1_amplitude'].set(440*18, min=0)#, max=440*2.5)
# controll sigma
pars_absorb_3_5cm['gauss_1_sigma'].set(20, min=1, max=68)

In [71]:
out_absorb_3_5cm = gauss_1.fit(blei_3_5cm.rate[1880:1993]-untergrund.rate[1880:1993],params=pars_absorb_3_5cm,x=x[1880:1993],weights=1/blei_3_5cm.error[1880:1993])

In [72]:
plt.errorbar(x,blei_3_5cm.rate-untergrund.rate, yerr=blei_3_5cm.error, fmt=' .g')
plt.plot(x,out_absorb_3_5cm.eval(x=x), '-r')
plt.show()

In [73]:
print(out_absorb_3_5cm.fit_report())

[[Model]]
    Model(gaussian, prefix='gauss_1_')
[[Fit Statistics]]
    # function evals   = 34
    # data points      = 113
    # variables        = 3
    chi-square         = 300.211
    reduced chi-square = 2.729
[[Variables]]
    gauss_1_center:      1919.46611 +/- 0.111573 (0.01%) (init= 1925)
    gauss_1_amplitude:   1.5077e+05 +/- 666.4545 (0.44%) (init= 7920)
    gauss_1_fwhm:        51.8710853 +/- 0.207614 (0.40%)  == '2.3548200*gauss_1_sigma'
    gauss_1_sigma:       22.0276222 +/- 0.088165 (0.40%) (init= 20)
[[Correlations]] (unreported correlations are <  0.100)
    C(gauss_1_center, gauss_1_sigma)  = -0.354 
    C(gauss_1_amplitude, gauss_1_sigma)  =  0.172 
    C(gauss_1_center, gauss_1_amplitude)  = -0.138 


### 2,5cm

In [74]:
pars_absorb_2_5cm = gauss_1.guess(blei_2_5cm.rate-untergrund.rate,x=x)
# controll the center
pars_absorb_2_5cm['gauss_1_center'].set(1925, min=1900, max=2000)
# controll the amplitude
pars_absorb_2_5cm['gauss_1_amplitude'].set(440*18, min=0)#, max=440*2.5)
# controll sigma
pars_absorb_2_5cm['gauss_1_sigma'].set(20, min=1, max=68)

In [75]:
out_absorb_2_5cm = gauss_1.fit(blei_2_5cm.rate[1890:1973]-untergrund.rate[1890:1973],params=pars_absorb_2_5cm,x=x[1890:1973],weights=1/blei_2_5cm.error[1890:1973])

In [76]:
plt.errorbar(x,blei_2_5cm.rate-untergrund.rate, yerr=blei_2_5cm.error, fmt=' .g')
plt.plot(x,out_absorb_2_5cm.eval(x=x), '-r')
plt.show()

In [77]:
print(out_absorb_2_5cm.fit_report())

[[Model]]
    Model(gaussian, prefix='gauss_1_')
[[Fit Statistics]]
    # function evals   = 43
    # data points      = 83
    # variables        = 3
    chi-square         = 334.888
    reduced chi-square = 4.186
[[Variables]]
    gauss_1_center:      1922.18608 +/- 0.093865 (0.00%) (init= 1925)
    gauss_1_amplitude:   4.7314e+05 +/- 1.63e+03 (0.34%) (init= 7920)
    gauss_1_fwhm:        53.0473251 +/- 0.209876 (0.40%)  == '2.3548200*gauss_1_sigma'
    gauss_1_sigma:       22.5271252 +/- 0.089126 (0.40%) (init= 20)
[[Correlations]] (unreported correlations are <  0.100)
    C(gauss_1_center, gauss_1_sigma)  = -0.430 
    C(gauss_1_amplitude, gauss_1_sigma)  =  0.408 
    C(gauss_1_center, gauss_1_amplitude)  = -0.290 


#### Exponentieller Zusammenhang Blei

In [78]:
blei_dicke = np.array([7.06,6.031,5.03,4.527,3.515,2.5])
blei_rate = np.array([2284,8300,27339,48414,1.5077e+05,4.7314e+05])
blei_rate = blei_rate*2
blei_dicke_err = np.array([0.01,0.009,0.007,0.009,0.007,0.001])
blei_rate_err = np.array([77,101,220,352,666,1630])
blei_rate_err = blei_rate*((blei_rate_err/blei_rate)**2+(2*0.02)**2+(0.1/300)**2)**0.5

In [79]:
blei_rate

array([   4568.,   16600.,   54678.,   96828.,  301540.,  946280.])

In [80]:
blei_rate_err

array([   198.2874603 ,    671.66034406,   2198.23249031,   3889.21640881,
        12080.39128051,  37887.59342347])

In [81]:
plt.errorbar(blei_dicke,blei_rate,yerr=blei_rate_err,xerr=blei_dicke_err,fmt=' .')
plt.show()

In [82]:
mod_exp_blei = Model(exp_fit)

In [83]:
pars_exp_blei = mod_exp_blei.make_params(a=1,b=1)

In [84]:
out_exp_blei = mod_exp_blei.fit(blei_rate,params=pars_exp_blei,x=blei_dicke,weights=1/(blei_rate_err))

In [85]:
plt.errorbar(blei_dicke,blei_rate,yerr=blei_rate_err,xerr=blei_dicke_err,fmt=' h', label="Messdaten",markersize=5)
plt.plot(np.arange(2,8,0.01),out_exp_blei.eval(x=np.arange(2,8,0.01)), label="Fit")
plt.xlabel("Dicke / cm",fontsize=24)
plt.ylabel("Counts",fontsize=24)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.xlim([2.4,7.3])
plt.ylim([0,10**6])
plt.ticklabel_format(style='sci', axis='y', scilimits=(-5,5))
plt.legend(loc='best',fontsize=24)
plt.show()

In [86]:
print(out_exp_blei.fit_report())

[[Model]]
    Model(exp_fit)
[[Fit Statistics]]
    # function evals   = 150
    # data points      = 6
    # variables        = 2
    chi-square         = 6.555
    reduced chi-square = 1.639
[[Variables]]
    a:   1.8132e+07 +/- 1.24e+06 (6.83%) (init= 1)
    b:   1.16441337 +/- 0.013773 (1.18%) (init= 1)
[[Correlations]] (unreported correlations are <  0.100)
    C(a, b)                      =  0.950 


## Aluminium

In [87]:
alu_5cm = pd.read_csv("Messdaten/absorbtion/absorbtion_2,5cmBlei+5cmAlu_LLDhoch.csv", header=14, delim_whitespace=True, names=['rate','error'], index_col=0)
alu_4cm = pd.read_csv("Messdaten/absorbtion/absorbtion_2,5cmBlei+4cmAlu_LLDhoch.csv", header=14, delim_whitespace=True, names=['rate','error'], index_col=0)
alu_3cm = pd.read_csv("Messdaten/absorbtion/absorbtion_2,5cmBlei+3cmAlu_LLDhoch.csv", header=14, delim_whitespace=True, names=['rate','error'], index_col=0)
alu_2cm = pd.read_csv("Messdaten/absorbtion/absorbtion_2,5cmBlei+2cmAlu_LLDhoch.csv", header=14, delim_whitespace=True, names=['rate','error'], index_col=0)
alu_1cm = pd.read_csv("Messdaten/absorbtion/absorbtion_2,5cmBlei+1cmAlu_LLDhoch.csv", header=14, delim_whitespace=True, names=['rate','error'], index_col=0)

### 5cm

In [88]:
pars_absorb_alu_5cm = gauss_1.guess(alu_5cm.rate-untergrund.rate,x=x)
# controll the center
pars_absorb_alu_5cm['gauss_1_center'].set(1925, min=1900, max=2000)
# controll the amplitude
pars_absorb_alu_5cm['gauss_1_amplitude'].set(440*18, min=0)#, max=440*2.5)
# controll sigma
pars_absorb_alu_5cm['gauss_1_sigma'].set(20, min=1, max=68)

In [89]:
out_absorb_alu_5cm = gauss_1.fit(alu_5cm.rate[1890:1993]-untergrund.rate[1890:1993],params=pars_absorb_alu_5cm,x=x[1890:1993],weights=1/alu_5cm.error[1890:1993])

In [90]:
plt.errorbar(x,alu_5cm.rate-untergrund.rate, yerr=alu_5cm.error, fmt=' .g')
plt.plot(x,out_absorb_alu_5cm.eval(x=x), '-r')
plt.show()

In [91]:
print(out_absorb_alu_5cm.fit_report())

[[Model]]
    Model(gaussian, prefix='gauss_1_')
[[Fit Statistics]]
    # function evals   = 34
    # data points      = 103
    # variables        = 3
    chi-square         = 218.822
    reduced chi-square = 2.188
[[Variables]]
    gauss_1_center:      1919.58544 +/- 0.113182 (0.01%) (init= 1925)
    gauss_1_amplitude:   1.7524e+05 +/- 705.9591 (0.40%) (init= 7920)
    gauss_1_fwhm:        51.3341155 +/- 0.203390 (0.40%)  == '2.3548200*gauss_1_sigma'
    gauss_1_sigma:       21.7995921 +/- 0.086371 (0.40%) (init= 20)
[[Correlations]] (unreported correlations are <  0.100)
    C(gauss_1_center, gauss_1_sigma)  = -0.554 
    C(gauss_1_amplitude, gauss_1_sigma)  =  0.355 
    C(gauss_1_center, gauss_1_amplitude)  = -0.344 


### 4cm

In [92]:
pars_absorb_alu_4cm = gauss_1.guess(alu_4cm.rate-untergrund.rate,x=x)
# controll the center
pars_absorb_alu_4cm['gauss_1_center'].set(1925, min=1900, max=2000)
# controll the amplitude
pars_absorb_alu_4cm['gauss_1_amplitude'].set(440*18, min=0)#, max=440*2.5)
# controll sigma
pars_absorb_alu_4cm['gauss_1_sigma'].set(20, min=1, max=68)

In [93]:
out_absorb_alu_4cm = gauss_1.fit(alu_4cm.rate[1890:1993]-untergrund.rate[1890:1993],params=pars_absorb_alu_4cm,x=x[1890:1993],weights=1/alu_4cm.error[1890:1993])

In [94]:
plt.errorbar(x,alu_4cm.rate-untergrund.rate, yerr=alu_4cm.error, fmt=' .g')
plt.plot(x,out_absorb_alu_4cm.eval(x=x), '-r')
plt.show()

In [95]:
print(out_absorb_alu_4cm.fit_report())

[[Model]]
    Model(gaussian, prefix='gauss_1_')
[[Fit Statistics]]
    # function evals   = 34
    # data points      = 103
    # variables        = 3
    chi-square         = 175.467
    reduced chi-square = 1.755
[[Variables]]
    gauss_1_center:      1919.59185 +/- 0.091328 (0.00%) (init= 1925)
    gauss_1_amplitude:   2.1317e+05 +/- 696.5389 (0.33%) (init= 7920)
    gauss_1_fwhm:        51.2655371 +/- 0.165897 (0.32%)  == '2.3548200*gauss_1_sigma'
    gauss_1_sigma:       21.7704695 +/- 0.070450 (0.32%) (init= 20)
[[Correlations]] (unreported correlations are <  0.100)
    C(gauss_1_center, gauss_1_sigma)  = -0.549 
    C(gauss_1_amplitude, gauss_1_sigma)  =  0.354 
    C(gauss_1_center, gauss_1_amplitude)  = -0.342 


### 3cm

In [96]:
pars_absorb_alu_3cm = gauss_1.guess(alu_3cm.rate-untergrund.rate,x=x)
# controll the center
pars_absorb_alu_3cm['gauss_1_center'].set(1925, min=1900, max=2000)
# controll the amplitude
pars_absorb_alu_3cm['gauss_1_amplitude'].set(440*18, min=0)#, max=440*2.5)
# controll sigma
pars_absorb_alu_3cm['gauss_1_sigma'].set(20, min=1, max=68)

In [97]:
out_absorb_alu_3cm = gauss_1.fit(alu_3cm.rate[1890:1993]-untergrund.rate[1890:1993],params=pars_absorb_alu_3cm,x=x[1890:1993],weights=1/alu_3cm.error[1890:1993])

In [98]:
plt.errorbar(x,alu_3cm.rate-untergrund.rate, yerr=alu_3cm.error, fmt=' .g')
plt.plot(x,out_absorb_alu_3cm.eval(x=x), '-r')
plt.show()

In [99]:
print(out_absorb_alu_3cm.fit_report())

[[Model]]
    Model(gaussian, prefix='gauss_1_')
[[Fit Statistics]]
    # function evals   = 34
    # data points      = 103
    # variables        = 3
    chi-square         = 193.138
    reduced chi-square = 1.931
[[Variables]]
    gauss_1_center:      1919.43538 +/- 0.088172 (0.00%) (init= 1925)
    gauss_1_amplitude:   2.6216e+05 +/- 815.6255 (0.31%) (init= 7920)
    gauss_1_fwhm:        51.6939967 +/- 0.159159 (0.31%)  == '2.3548200*gauss_1_sigma'
    gauss_1_sigma:       21.9524196 +/- 0.067588 (0.31%) (init= 20)
[[Correlations]] (unreported correlations are <  0.100)
    C(gauss_1_center, gauss_1_sigma)  = -0.558 
    C(gauss_1_amplitude, gauss_1_sigma)  =  0.363 
    C(gauss_1_center, gauss_1_amplitude)  = -0.354 


### 2cm

In [100]:
pars_absorb_alu_2cm = gauss_1.guess(alu_2cm.rate-untergrund.rate,x=x)
# controll the center
pars_absorb_alu_2cm['gauss_1_center'].set(1925, min=1900, max=2000)
# controll the amplitude
pars_absorb_alu_2cm['gauss_1_amplitude'].set(440*18, min=0)#, max=440*2.5)
# controll sigma
pars_absorb_alu_2cm['gauss_1_sigma'].set(20, min=1, max=68)

In [101]:
out_absorb_alu_2cm = gauss_1.fit(alu_2cm.rate[1890:1993]-untergrund.rate[1890:1993],params=pars_absorb_alu_2cm,x=x[1890:1993],weights=1/alu_2cm.error[1890:1993])

In [102]:
plt.errorbar(x,alu_2cm.rate-untergrund.rate, yerr=alu_2cm.error, fmt=' .g')
plt.plot(x,out_absorb_alu_2cm.eval(x=x), '-r')
plt.show()

In [103]:
print(out_absorb_alu_2cm.fit_report())

[[Model]]
    Model(gaussian, prefix='gauss_1_')
[[Fit Statistics]]
    # function evals   = 34
    # data points      = 103
    # variables        = 3
    chi-square         = 231.897
    reduced chi-square = 2.319
[[Variables]]
    gauss_1_center:      1919.61995 +/- 0.087786 (0.00%) (init= 1925)
    gauss_1_amplitude:   3.1760e+05 +/- 982.1848 (0.31%) (init= 7920)
    gauss_1_fwhm:        51.8127210 +/- 0.159185 (0.31%)  == '2.3548200*gauss_1_sigma'
    gauss_1_sigma:       22.0028371 +/- 0.067599 (0.31%) (init= 20)
[[Correlations]] (unreported correlations are <  0.100)
    C(gauss_1_center, gauss_1_sigma)  = -0.558 
    C(gauss_1_amplitude, gauss_1_sigma)  =  0.362 
    C(gauss_1_center, gauss_1_amplitude)  = -0.351 


### 1cm

In [104]:
pars_absorb_alu_1cm = gauss_1.guess(alu_1cm.rate-untergrund.rate,x=x)
# controll the center
pars_absorb_alu_1cm['gauss_1_center'].set(1925, min=1900, max=2000)
# controll the amplitude
pars_absorb_alu_1cm['gauss_1_amplitude'].set(440*18, min=0)#, max=440*2.5)
# controll sigma
pars_absorb_alu_1cm['gauss_1_sigma'].set(20, min=1, max=68)

In [105]:
out_absorb_alu_1cm = gauss_1.fit(alu_1cm.rate[1870:2000]-untergrund.rate[1870:2000],params=pars_absorb_alu_1cm,x=x[1870:2000],weights=1/alu_1cm.error[1870:2000])

In [106]:
plt.errorbar(x,alu_1cm.rate-untergrund.rate, yerr=alu_1cm.error, fmt=' .g', label="Messwerte")
plt.plot(x,out_absorb_alu_1cm.eval(x=x), '-r', label="Fit")
plt.xlabel("Kanal",fontsize=24)
plt.ylabel("Counts",fontsize=24)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.ticklabel_format(style='sci', axis='y', scilimits=(-5,5))
plt.legend(loc='best',fontsize=24)
plt.show()

In [107]:
print(out_absorb_alu_1cm.fit_report())

[[Model]]
    Model(gaussian, prefix='gauss_1_')
[[Fit Statistics]]
    # function evals   = 33
    # data points      = 130
    # variables        = 3
    chi-square         = 1106.817
    reduced chi-square = 8.715
[[Variables]]
    gauss_1_center:      1918.61200 +/- 0.117355 (0.01%) (init= 1925)
    gauss_1_amplitude:   3.9310e+05 +/- 1.88e+03 (0.48%) (init= 7920)
    gauss_1_fwhm:        53.7107214 +/- 0.213568 (0.40%)  == '2.3548200*gauss_1_sigma'
    gauss_1_sigma:       22.8088437 +/- 0.090694 (0.40%) (init= 20)
[[Correlations]] (unreported correlations are <  0.100)
    C(gauss_1_center, gauss_1_sigma)  = -0.247 


#### Exponentieller Zusammenhang Alu

In [108]:
alu_dicke = np.array([7.6025,6.585,5.57,4.55,3.545])
alu_rate = np.array([1.7524e+05,2.1317e+05,2.6216e+05,3.1760e+05,3.9311e+05])
alu_rate = alu_rate*2
alu_dicke_err = np.array([0.002,0.002,0.002,0.001,0.001])
alu_rate_err = np.array([705,696,815,982,1.42e+03])
alu_rate_err = alu_rate*((alu_rate_err/alu_rate)**2+(2*0.02)**2+(0.1/300)**2)**0.5

In [109]:
alu_rate

array([ 350480.,  426340.,  524320.,  635200.,  786220.])

In [110]:
alu_rate_err

array([ 14037.40154409,  17068.38847576,  20989.35707824,  25427.85124631,
        31481.93297547])

In [111]:
mod_exp_alu = Model(exp_fit)

In [112]:
pars_exp_alu = mod_exp_alu.make_params(a=1,b=1)

In [113]:
out_exp_alu = mod_exp_alu.fit(alu_rate,params=pars_exp_alu,x=alu_dicke,weights=1/(alu_rate_err))

In [114]:
plt.errorbar(alu_dicke,alu_rate,yerr=alu_rate_err,xerr=alu_dicke_err,fmt=' .', label="Messdaten")
plt.plot(np.arange(3,8,0.01),out_exp_alu.eval(x=np.arange(3,8,0.01)), label="Fit")
plt.xlabel("Dicke / cm",fontsize=24)
plt.ylabel("Rate / Hz",fontsize=24)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.xlim([3.4,7.7])
#plt.ylim([1.5*10**5,4.5*10**5])
plt.ticklabel_format(style='sci', axis='y', scilimits=(-5,5))
plt.legend(loc='best',fontsize=24)
plt.show()

In [115]:
print(out_exp_alu.fit_report())

[[Model]]
    Model(exp_fit)
[[Fit Statistics]]
    # function evals   = 135
    # data points      = 5
    # variables        = 2
    chi-square         = 0.077
    reduced chi-square = 0.026
[[Variables]]
    a:   1.5798e+06 +/- 1.82e+04 (1.15%) (init= 1)
    b:   0.19845671 +/- 0.002001 (1.01%) (init= 1)
[[Correlations]] (unreported correlations are <  0.100)
    C(a, b)                      =  0.969 


#### Bestimmen der Counts ohne Material vor dem Detektor

In [116]:
out_exp_blei.eval(x=np.arange(0,1))*300

array([  5.43972331e+09])

In [117]:
(1-0.116/0.120)*100

3.3333333333333215

In [118]:
(1-0.0198/0.0190)*100

-4.210526315789487

In [119]:
Wirkungsquerschnitt20GradAlu = pd.read_csv("/home/arrow13/Git/FP/Compton/Messdaten/Grad/Messung_1cmAlu_20Grad_LLDhoch.csv", header=14, delim_whitespace=True, names=['rate','error'], index_col=0)
Wirkungsquerschnitt30GradAlu = pd.read_csv("/home/arrow13/Git/FP/Compton/Messdaten/Grad/Messung_1cmAlu_30Grad_LLDhoch.csv", header=14, delim_whitespace=True, names=['rate','error'], index_col=0)
Wirkungsquerschnitt40GradAlu = pd.read_csv("/home/arrow13/Git/FP/Compton/Messdaten/Grad/Messung_1cmAlu_40Grad_LLDhoch.csv", header=14, delim_whitespace=True, names=['rate','error'], index_col=0)
Wirkungsquerschnitt50GradAlu = pd.read_csv("/home/arrow13/Git/FP/Compton/Messdaten/Grad/Messung_1cmAlu_50Grad_LLDhoch.csv", header=14, delim_whitespace=True, names=['rate','error'], index_col=0)
Wirkungsquerschnitt60GradAlu = pd.read_csv("/home/arrow13/Git/FP/Compton/Messdaten/Grad/Messung_1cmAlu_60Grad_LLDhoch.csv", header=14, delim_whitespace=True, names=['rate','error'], index_col=0)
Wirkungsquerschnitt70GradAlu = pd.read_csv("/home/arrow13/Git/FP/Compton/Messdaten/Grad/Messung_1cmAlu_70Grad_LLDhoch.csv", header=14, delim_whitespace=True, names=['rate','error'], index_col=0)
Wirkungsquerschnitt80GradAlu = pd.read_csv("/home/arrow13/Git/FP/Compton/Messdaten/Grad/Messung_1cmAlu_80Grad_LLDhoch.csv", header=14, delim_whitespace=True, names=['rate','error'], index_col=0)
Wirkungsquerschnitt20Grad = pd.read_csv("/home/arrow13/Git/FP/Compton/Messdaten/Grad/Messung_20Grad_LLDhoch.csv", header=14, delim_whitespace=True, names=['rate','error'], index_col=0)
Wirkungsquerschnitt30Grad = pd.read_csv("/home/arrow13/Git/FP/Compton/Messdaten/Grad/Messung_30Grad_LLDhoch.csv", header=14, delim_whitespace=True, names=['rate','error'], index_col=0)
Wirkungsquerschnitt40Grad = pd.read_csv("/home/arrow13/Git/FP/Compton/Messdaten/Grad/Messung_40Grad_LLDhoch.csv", header=14, delim_whitespace=True, names=['rate','error'], index_col=0)
Wirkungsquerschnitt50Grad = pd.read_csv("/home/arrow13/Git/FP/Compton/Messdaten/Grad/Messung_50Grad_LLDhoch.csv", header=14, delim_whitespace=True, names=['rate','error'], index_col=0)
Wirkungsquerschnitt60Grad = pd.read_csv("/home/arrow13/Git/FP/Compton/Messdaten/Grad/Messung_60Grad_LLDhoch.csv", header=14, delim_whitespace=True, names=['rate','error'], index_col=0)
Wirkungsquerschnitt70Grad = pd.read_csv("/home/arrow13/Git/FP/Compton/Messdaten/Grad/Messung_70Grad_LLDhoch.csv", header=14, delim_whitespace=True, names=['rate','error'], index_col=0)
Wirkungsquerschnitt80Grad = pd.read_csv("/home/arrow13/Git/FP/Compton/Messdaten/Grad/Messung_80Grad_LLDhoch.csv", header=14, delim_whitespace=True, names=['rate','error'], index_col=0)

In [120]:
Kaesekuchen1 = []
Kaesekuchen2 = []
Kaesekuchen1.append(Wirkungsquerschnitt20GradAlu)
Kaesekuchen1.append(Wirkungsquerschnitt30GradAlu)
Kaesekuchen1.append(Wirkungsquerschnitt40GradAlu)
Kaesekuchen1.append(Wirkungsquerschnitt50GradAlu)
Kaesekuchen1.append(Wirkungsquerschnitt60GradAlu)
Kaesekuchen1.append(Wirkungsquerschnitt70GradAlu)
Kaesekuchen1.append(Wirkungsquerschnitt80GradAlu)
Kaesekuchen2.append(Wirkungsquerschnitt20Grad)
Kaesekuchen2.append(Wirkungsquerschnitt30Grad)
Kaesekuchen2.append(Wirkungsquerschnitt40Grad)
Kaesekuchen2.append(Wirkungsquerschnitt50Grad)
Kaesekuchen2.append(Wirkungsquerschnitt60Grad)
Kaesekuchen2.append(Wirkungsquerschnitt70Grad)
Kaesekuchen2.append(Wirkungsquerschnitt80Grad)

In [121]:
for Kaesekrecker1,Kaesekrecker2 in zip(Kaesekuchen1,Kaesekuchen2):
    plt.errorbar(x,Kaesekrecker1.rate-Kaesekrecker2.rate,yerr=np.sqrt(Kaesekrecker1.error**2+Kaesekrecker2.error**2), fmt=' .g')
    plt.xlim([0,2050])
    plt.xlabel("Kanal",fontsize=24)
    plt.ylabel("Counts",fontsize=24)
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)
    plt.legend(loc='best',fontsize=24)
    plt.show()

/home/arrow13/anaconda3/lib/python3.4/site-packages/matplotlib/axes/_axes.py:475: UserWarning: No labelled objects found. Use label='...' kwarg on individual plots.
  warnings.warn("No labelled objects found. "


In [182]:
i = 0
Käsefit_Amplitude = np.array([15.5,15.25,15.5,15.5,15,15.5,15.5])
Käsefit_Center = np.array([1855,1775,1610,1460,1300,1160,1040])
Käsefit_Sigmamin = np.array([30,62,63,63,63,31,30])
Käsefit_YOben = np.array([165,100,69,63,58,55,67])
Käsefit_YUnten = np.array([-14,-12,-8,-7.5,-7.5,-8,-8])
Käsefit_PM = np.array([175,225,250,250,250,250,250])
Käsefit_Oben = Käsefit_Center+Käsefit_PM#np.array([2000,2000,1900,1800,1600,1400,1300])
Käsefit_Unten = Käsefit_Center-Käsefit_PM#np.array([1600,1600,1400,1200,1100,1000,800])
for Kaesekrecker1,Kaesekrecker2 in zip(Kaesekuchen1,Kaesekuchen2):
    fitdaten = (Kaesekrecker1.rate[Käsefit_Unten[i]:Käsefit_Oben[i]]-Kaesekrecker2.rate[Käsefit_Unten[i]:Käsefit_Oben[i]]+abs(Kaesekrecker1.rate[Käsefit_Unten[i]:Käsefit_Oben[i]]-Kaesekrecker2.rate[Käsefit_Unten[i]:Käsefit_Oben[i]]))/2
    parametersWQ20 = gauss_1.guess(fitdaten,x=x)
    parametersWQ20['gauss_1_sigma'].set(20, min=Käsefit_Sigmamin[i], max=65)
    parametersWQ20['gauss_1_amplitude'].set(440*Käsefit_Amplitude[i], min=0)#, max=440*2.5)
    parametersWQ20['gauss_1_center'].set(Käsefit_Center[i], min=Käsefit_Unten[i]+200, max=Käsefit_Oben[i]-200)
    antiWeights = (np.sqrt(Kaesekrecker1.error**2+Kaesekrecker2.error**2)+1+abs(np.sqrt(Kaesekrecker1.error**2+Kaesekrecker2.error**2)-1))/2
    out_parameterWQ20 = gauss_1.fit(fitdaten,params=parametersWQ20,x=x[Käsefit_Unten[i]:Käsefit_Oben[i]],weights=1/antiWeights)
    print(Käsefit_Unten[i],Käsefit_Oben[i])
    plt.errorbar(x,(Kaesekrecker1.rate-Kaesekrecker2.rate+1+abs(Kaesekrecker1.rate-Kaesekrecker2.rate-1))/2, yerr= np.sqrt(Kaesekrecker1.error**2+Kaesekrecker2.error**2), fmt=' .g', label="Messwerte")
    plt.plot(x,out_parameterWQ20.eval(x=x), '-r', label="Fit")
    plt.xlim(Käsefit_Unten[i],Käsefit_Oben[i])
    plt.ylim(Käsefit_YUnten[i],Käsefit_YOben[i])
    plt.xlabel("Kanal",fontsize=24)
    plt.ylabel("Counts",fontsize=24)
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)
    plt.ticklabel_format(style='sci', axis='y', scilimits=(-5,5))
    plt.legend(loc='best',fontsize=24)
    plt.show()
    print(out_parameterWQ20.fit_report())
    i = i+1



#pars_absorb_alu_1cm = gauss_1.guess(alu_1cm.rate-untergrund.rate,x=x)
# controll the center
#pars_absorb_alu_1cm['gauss_1_center'].set(1925, min=1900, max=2000)
# controll the amplitude
#pars_absorb_alu_1cm['gauss_1_amplitude'].set(440*18, min=0)#, max=440*2.5)
# controll sigma
#pars_absorb_alu_1cm['gauss_1_sigma'].set(20, min=1, max=68)

1680 2030
[[Model]]
    Model(gaussian, prefix='gauss_1_')
[[Fit Statistics]]
    # function evals   = 9
    # data points      = 350
    # variables        = 3
    chi-square         = 325.270
    reduced chi-square = 0.937
[[Variables]]
    gauss_1_center:      1843.21755 +/- 0.967289 (0.05%) (init= 1855)
    gauss_1_amplitude:   7671.67689 +/- 159.2324 (2.08%) (init= 6820)
    gauss_1_fwhm:        82.1537542 +/- 1.466469 (1.79%)  == '2.3548200*gauss_1_sigma'
    gauss_1_sigma:       34.8874879 +/- 0.622752 (1.79%) (init= 30)
[[Correlations]] (unreported correlations are <  0.100)
    C(gauss_1_center, gauss_1_amplitude)  =  0.393 
    C(gauss_1_center, gauss_1_sigma)  =  0.256 
    C(gauss_1_amplitude, gauss_1_sigma)  =  0.210 
1550 2000
[[Model]]
    Model(gaussian, prefix='gauss_1_')
[[Fit Statistics]]
    # function evals   = 17
    # data points      = 450
    # variables        = 3
    chi-square         = 486.512
    reduced chi-square = 1.088
[[Variables]]
    gauss_1_center: